# 完整指南：立即执行策略

按照以下 6 个简单步骤，运行在上文中“**完整Python代码**”部分展示的代码并立即查看回测结果。您无需编写任何代码 — 所有内容都已为您预设完成。

## 第一步：使用用户ID连接API

#### 这一步让你在执行交易算法并获取回测结果之前，安全地连接 QuantConnect API。

具体分解步骤如下：

1. 首先，系统会将你的 API 密钥与当前时间结合，生成一个时效性令牌，并进行加密以确保安全，然后将其格式化，以便通过网络发送到服务器。

2. 接着，它会使用该令牌发送API连接请求。

3. 最后，服务器会返回结果，确认连接是否成功。

为了安全起见， **`USER_ID`** 和 **`API_TOKEN`** 的数值已被隐藏。立即加入 QuantConnect 会员，获取您的专属用户 ID 和 API 密钥 — 开始执行您自己的策略！

In [2]:
from base64 import b64encode ## 将文本编码为 Base64
from hashlib import sha256 ## 一个将任意文本“搅碎”成混乱字符串的“搅拌机”（出于安全考量）
from time import time ## 获取当前时间（以秒为单位），用于生成唯一的时间戳
from requests import get, post ## requests：用于发送 HTTP 请求；get：用于从服务器获取数据；post：用于向服务器发送数据

import json ## 以 4 个空格缩进的方式美观地打印来自 API 的 JSON 结果
BASE_URL = 'https://www.quantconnect.com/api/v2/'

# 你可以在 https://www.quantconnect.com/organization/找到组织 ID。（不必需） 
USER_ID = 224901
API_TOKEN = 'b26bdf0c94229806b8515308d77f75b6aa9829d2802540de0c9e9d0dc477da4e'
## ORGANIZATION_ID = '____'

C:\ProgramData\anaconda3\Lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.4.0) or chardet (4.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


In [5]:
def get_headers(): ## 返回发送安全 API 请求所需的请求头信息
    
    # 获取时间戳
    timestamp = f'{int(time())}' ## 获取当前时间（例如“1722458765”）。这样可以确保令牌只在短时间内有效，增加安全性。
    time_stamped_token = f'{API_TOKEN}:{timestamp}'.encode('utf-8') ## 将 API 令牌与当前时间用冒号分隔组合，如 b26bd...da4e:1722458765，然后使用 UTF-8 编码转换成计算机可读的字节

    # 获取哈希后的 API 令牌
    hashed_token = sha256(time_stamped_token).hexdigest() ## 通过 SHA-256 哈希算法处理，生成一串长且混乱的字符串，无法被逆向破解

    authentication = f'{USER_ID}:{hashed_token}'.encode('utf-8') ## 将用户 ID 与哈希令牌组合后编码成字节，这就是加密形式的登录身份
    authentication = b64encode(authentication).decode('ascii') ## 对编码后的身份信息进行 Base64 编码，以便安全地通过互联网传输。然后将其解码为可读的文本字符串。

    # 创建请求头字典
    return {
        'Authorization': f'Basic {authentication}', ## 你的 Base64 登录字符串
        'Timestamp': timestamp ## 你的 Base64 登录字符串
    } ## 这就是你在 API 请求中用来证明身份的方式

# 登录系统以验证你的身份
response = post(f'{BASE_URL}/authenticate', headers = get_headers()) ## 'post' 函数向 API 发送数据；它向网址 [www.quantconnect.com/api/v2/authenticate](http://www.quantconnect.com/api/v2/authenticate) 发送请求；请求中包含通过你的 get\_headers() 函数生成的登录信息头
print("身份验证结果：")
print(response.json()) ## POST 请求的响应保存在变量 “response” 中；用于显示身份验证是否成功

# --------------------

# 要管理的项目 ID
project_id = 0 ## 在下一步中替换为已创建的项目 ID。

身份验证结果：
{'success': True}


## 第二步：创建新项目

这一步通过 API 在 QuantConnect 上创建一个新的**项目（project）**——每个算法都必须存在于项目内。你可以把项目理解为你交付内容时的一个包装盒。

**输入**包括：

1. 项目名称，
2. 使用的编程语言（“Py”代表 Python）。

然后，它会将这些信息发送到 `/projects/create` API 端点。

**输出**会返回新项目的唯一 ID。

In [8]:
### 创建项目
# 向 `/projects/create` 端点发送 POST 请求以创建新项目
response = post(f'{BASE_URL}/projects/create', headers=get_headers(), json={  ## json={...} 部分包含你发送的项目信息——一个包含两个条目的字典：项目名称和编程语言
    "name": f"Project_{int(time())}",  # 使用当前时间戳生成唯一的项目名称
    "language": "Py"  # 项目的编程语言（Python）
})

# 将 JSON 响应解析为 Python 可操作的字典
result = response.json() ## json() 将 JSON 响应转换为字典 —— 而不是相反的操作

# 从返回字节中提取项目 ID。
project_id = result['projects'][0]['projectId'] ## “result” 是一个嵌套字典：第一层有两个键：'success' 和 'projects'，其中 'projects' 下又包含两个键：'projectId' 和 'name'
## \[0] 这里表示访问 'projects' 列表中的第一个字典，该字典包含键 'projectId'

# 检查请求是否成功，并打印结果
if result['success']:
    print("项目创建成功。你的项目ID是 " + str(project_id) + " 。\n")
    ## print(result)
    print(json.dumps(result, indent=4))

项目创建成功。你的项目ID是 23814211 。

{
    "projects": [
        {
            "projectId": 23814211,
            "organizationId": "0aa48a67d71e36c6a3870ebb8c224664",
            "name": "Project_1751550471",
            "modified": "2025-07-03 13:47:52",
            "created": "2025-07-03 13:47:52",
            "ownerId": 224901,
            "language": "Py",
            "collaborators": [
                {
                    "uid": 224901,
                    "liveControl": true,
                    "permission": "write",
                    "publicId": "haixiang-yan",
                    "profileImage": "https://cdn.quantconnect.com/web/i/users/profile/4f9b4ae886ebdb31674640077.png",
                    "email": "hxyan.2015@gmail.com",
                    "name": "Haixiang Yan",
                    "bio": null,
                    "owner": true
                }
            ],
            "leanVersionId": 17170,
            "leanPinnedToMaster": true,
            "owner": true,
            

## 第三步：创建算法文件

这一步通过 API 将**算法文件**放入你在步骤 2 中定义的 QuantConnect 项目（“包装盒”）中。

**输入**包括：

1. 你定义的文件名（例如 `utils.py`），
2. 文件内容（即交易算法），
3. 关联的项目 ID（“包装盒”）。

然后，它会将这些信息发送到 QuantConnect 的 `/files/create` 端点。

**输出**会确认上传成功并返回上传的文件名。

In [9]:
### 创建文件
import json
import os

# 定义文件的相对路径
file_path = os.path.join("algos", "utils.py")

# 从子文件夹中读取代码内容
with open(file_path, "r", encoding="utf-8") as file:
    code_content = file.read()

# 向 `/files/create` 端点发送 POST 请求以创建新文件
response = post(f'{BASE_URL}/files/create', headers=get_headers(), json={
    "projectId": project_id,  # 项目 ID
    "name": "utils.py",  # 算法文件名
    "content": code_content  # 文件内容（Python算法）
})

# 将 JSON 响应解析为 Python 可操作的字典
result = response.json()
## print(result)

# 检查请求是否成功，并打印结果
if result['success']:
    files = result.get("files", [])
    if files and isinstance(files, list):
        file_name = files[0].get("name", "Unknown File")
    else:
        file_name = "Unknown File"

    print("算法文件创建成功。你的文件名是 " + file_name + " 。\n")
    ## print(result)
    print(json.dumps(result, indent=4))

算法文件创建成功。你的文件名是 utils.py

{
    "files": [
        {
            "id": 96001233,
            "name": "utils.py",
            "content": "#!/usr/bin/env python\n# coding: utf-8\n\n# In[ ]:\n\n\nfrom AlgorithmImports import *\n\nclass JanuaryEffectInStocks(QCAlgorithm):\n\n    def Initialize(self):\n        self.SetStartDate(2000, 1, 1)  \n        self.SetCash(100000) \n\n        data = self.AddEquity(\"SPY\", Resolution.Daily)\n        data.SetLeverage(10)\n        self.large_cap = data.Symbol\n        \n        data = self.AddEquity(\"IWM\", Resolution.Daily)\n        data.SetLeverage(10)\n        self.small_cap = data.Symbol\n\n        self.start_price = None\n        self.recent_month = -1\n        \n    def OnData(self, data):\n        if self.recent_month == self.Time.month:\n            return\n        self.recent_month = self.Time.month\n\n        if self.Securities[self.large_cap].GetLastData() and self.Securities[self.small_cap].GetLastData():\n            if (self.Time.date() 

## 第四步：创建编译任务

这一步向 QuantConnect 发送请求，**编译**已上传的算法文件。

**输入**是 `projectId`，用于告诉系统要编译哪个项目。

**输出**确认编译成功，并打印 `compileId`，表示你的代码已成功编译，准备好运行回测。

In [10]:
### 创建编译任务
# 准备数据载荷以创建编译任务
payload = {
    "projectId": project_id  # 需要编译的项目 ID
}

# 向 `/compile/create` 端点发送 POST 请求以开始编译
response = post(f'{BASE_URL}/compile/create', headers=get_headers(), json=payload)

# 将 JSON 响应解析为 Python 可操作的字典
result = response.json()

# 从响应中提取编译 ID
compile_id = result['compileId']

# 检查请求是否成功，并打印结果
if result['success']:
    print("编译任务创建成功。你的编译ID是 " + str(compile_id) + " 。\n")
    print(result)
    ## print(result)
    print(json.dumps(result, indent=4))

编译任务创建成功。你的编译ID是 88358864bff68bd1670b271edd4ec36e-8321e6cf7a315a90351d2e46ffb84f94 。

{'compileId': '88358864bff68bd1670b271edd4ec36e-8321e6cf7a315a90351d2e46ffb84f94', 'state': 'InQueue', 'parameters': [], 'projectId': '23814211', 'signature': '8321e6cf7a315a90351d2e46ffb84f94', 'signatureOrder': ['project/main.py', 'project/utils.py'], 'success': True}
{
    "compileId": "88358864bff68bd1670b271edd4ec36e-8321e6cf7a315a90351d2e46ffb84f94",
    "state": "InQueue",
    "parameters": [],
    "projectId": "23814211",
    "signature": "8321e6cf7a315a90351d2e46ffb84f94",
    "signatureOrder": [
        "project/main.py",
        "project/utils.py"
    ],
    "success": true
}


## 第五步：开始回测

这一步在 QuantConnect 上为你的交易算法创建一个**回测项目**。

**输入**包括：

1. 你的 `projectId`，
2. 被成功执行的 `compileId`，
3. 回测名称。

然后，它会将这些输入发送到 `/backtests/create` API端点。

**输出**是一条确认信息，包含唯一的 `backtestId`，表示你的策略正在基于历史市场数据运行测试。请注意，回测完成时间取决于算法的复杂程度，可能从几秒到几分钟不等。

In [11]:
### 创建回测项目
# 向 `/backtests/create` 端点发送 POST 请求以创建回测
response = post(f'{BASE_URL}/backtests/create', headers=get_headers(), json={
    "projectId": project_id,  # 需要回测的项目 ID
    "compileId": compile_id,  # 需要回测的编译 ID
    "backtestName": f"Backtest {int(time())}"  # 使用当前时间戳生成回测的唯一名称
})

# 将 JSON 响应解析为 Python 可操作的字典
result = response.json()

# 从响应中提取回测 ID
backtest_id = result['backtest']['backtestId']

# 检查请求是否成功，并打印结果
if result['success']:
    print("太棒了，回测执行成功。你的回测ID是 " + str(backtest_id) + " 。\n")
    ## print(result)
    print(json.dumps(result, indent=4))
    print(backtest_id)

太棒了，回测执行成功。你的回测ID是 cd38a15570c7bd7f285f7d8618c3a390 。

{
    "backtest": {
        "note": null,
        "name": "Backtest 1751550618",
        "organizationId": 218385,
        "projectId": 23814211,
        "completed": false,
        "optimizationId": null,
        "backtestId": "cd38a15570c7bd7f285f7d8618c3a390",
        "tradeableDates": 0,
        "researchGuide": {
            "minutes": 1,
            "backtestCount": 1,
            "parameters": 11
        },
        "backtestStart": "2025-07-03 13:50:19",
        "backtestEnd": "2025-07-03 13:50:19",
        "created": "2025-07-03 13:50:19",
        "snapshotId": 23814256,
        "status": "In Queue...",
        "error": null,
        "stacktrace": null,
        "progress": 0,
        "hasInitializeError": false,
        "charts": {
            "Strategy Equity": {
                "name": "Strategy Equity"
            },
            "Benchmark": {
                "name": "Benchmark"
            },
            "Portfolio Marg

## 第六步：读取回测表现

最后这一步用于获取并展示 QuantConnect 上已完成回测的表现。

**输入**包括 `projectId` 和 `backtestId`。

随后，它会将这些信息发送到 `/backtests/read` 端点。

**输出**是一个表格，显示如夏普比率（Sharpe Ratio）、年化收益率、回撤幅度和年化收益率等指标。这些结果有助于评估你的交易策略表现如何。

In [12]:
### 读取回测表现
## 可能需要稍等几分钟以获取回测结果
# 准备数据载荷以读取回测表现
import json
from tabulate import tabulate

payload = {
    "projectId": project_id,  # 项目 ID
    "backtestId": backtest_id  # 回测 ID
}

# 向 /backtests/read 端点发送 POST 请求以获取回测数据
response = post(f'{BASE_URL}/backtests/read', headers=get_headers(), json=payload)

# 将 JSON 响应解析为 Python 可操作的字典
result = response.json()

# 检查请求是否成功，并打印回测表现数据
if result['success']:
    print("回测表现")
    backtestInfo = result.get("backtest",{})
    stats = result.get("backtest",{}).get("totalPerformance",{}).get("portfolioStatistics", {})
    ## print(stats)
    metrics = {
        "交易执行天数": backtestInfo.get("tradeableDates", "N/A"),
        "回测开始时间": backtestInfo.get("backtestStart", "N/A"),
        "回测结束时间": backtestInfo.get("backtestEnd", "N/A"),
        "当前资产余额": (
            f"{float(stats.get('endEquity')):.2f}"
            if stats.get("endEquity") not in [None, "N/A"]
            else "N/A"
        ),
        "复合年化收益率": (
            f"{float(stats.get('compoundingAnnualReturn')) * 100:.2f}%"
            if stats.get('compoundingAnnualReturn') not in [None, "N/A"]
            else "N/A"
        ),
        "回撤幅度": (
            f"{float(stats.get('drawdown')) * 100:.2f}%"
            if stats.get('drawdown') not in [None, "N/A"]
            else "N/A"
        ),
        "总净利润": (
            f"{float(stats.get('totalNetProfit')) * 100:.2f}%"
            if stats.get('totalNetProfit') not in [None, "N/A"]
            else "N/A"
        ),
        "夏普比率": (
            f"{float(stats.get('sharpeRatio')):.2f}"
            if stats.get("sharpeRatio") not in [None, "N/A"]
            else "N/A"
        ),
        "索提诺比率": (
            f"{float(stats.get('sortinoRatio')):.2f}"
            if stats.get("sortinoRatio") not in [None, "N/A"]
            else "N/A"
        ),
        "特雷诺比率": (
            f"{float(stats.get('treynorRatio')):.2f}"
            if stats.get("treynorRatio") not in [None, "N/A"]
            else "N/A"
        ),
        "贝塔": (
            f"{float(stats.get('beta')):.2f}"
            if stats.get("beta") not in [None, "N/A"]
            else "N/A"
        ),
        "信息比率": (
            f"{float(stats.get('informationRatio')):.2f}"
            if stats.get("informationRatio") not in [None, "N/A"]
            else "N/A"
        ),
        "跟踪误差": (
            f"{float(stats.get('trackingError')) * 100:.2f}%"
            if stats.get('trackingError') not in [None, "N/A"]
            else "N/A"
        ),
        "投资组合周转率": (
            f"{float(stats.get('portfolioTurnover')) * 100:.2f}%"
            if stats.get('portfolioTurnover') not in [None, "N/A"]
            else "N/A"
        ),
        "VaR 99": stats.get("valueAtRisk99", "N/A"),
        "VaR 95": stats.get("valueAtRisk95", "N/A")
    }

    print(tabulate(metrics.items(), headers=["指标", "数值"], tablefmt="grid"))

# print the entire backtest statistics

## if result['success']:
    ## print("Backtest Statistics:")
    ### print(result)
    ## print(json.dumps(result, indent=4)) 

回测表现
+----------------+---------------------+
| 指标           | 数值                |
+================+=====================+
| 交易执行天数   | 376                 |
+----------------+---------------------+
| 回测开始时间   | 2024-01-02 00:00:00 |
+----------------+---------------------+
| 回测结束时间   | 2025-07-02 23:59:59 |
+----------------+---------------------+
| 当前资产余额   | 117412.83           |
+----------------+---------------------+
| 复合年化收益率 | 11.29%              |
+----------------+---------------------+
| 回撤幅度       | 17.90%              |
+----------------+---------------------+
| 总净利润       | 17.41%              |
+----------------+---------------------+
| 夏普比率       | 0.26                |
+----------------+---------------------+
| 索提诺比率     | 0.28                |
+----------------+---------------------+
| 特雷诺比率     | 0.04                |
+----------------+---------------------+
| 贝塔           | 0.74                |
+----------------+---------------------+
| 信息比率       | -0.97         

### 恭喜您成功执行一条策略并获得回测表现数据！